In [5]:
import tensorflow as tf
import gensim

In [11]:
import numpy as np

In [24]:
import pandas as pd

In [25]:
path_to_data = "/Users/anne/repos/AEM/data/dataset_vermeer.pkl"

In [26]:
df = pd.read_pickle(path_to_data)
data = list(df['text'])
labels = list(df['topic'])

In [32]:
data[2]

'\n De dominee van Hillary Clinton brengt volgende week een boek uit: Strong for a moment like this: een bundeling van stichtelijke teksten die deze Bill Shillady haar elke dag tijdens haar laatste presidentiële campagne stuurde. Volgens Hillary Clinton zelf begon ze elke dag met een gemailde...'

In [53]:
t.fit_on_texts(data)
vocab_size = len(t.word_index) + 1

In [54]:
encoded_docs = t.texts_to_sequences(data)
print(encoded_docs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [58]:
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 320))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

In [28]:
t = tf.keras.preprocessing.text.Tokenizer()

In [59]:
raw_embedding = load_embedding('/Users/anne/repos/embedding_models/test/cow-320.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, t.word_index)

In [52]:
type(embedding_vectors)

numpy.ndarray

In [39]:
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=4, trainable=False)

In [93]:
model = tf.keras.Sequential()
e = tf.keras.layers.Embedding(vocab_size, 320, weights=[embedding_vectors], input_length=4, trainable=False)
model.add(e)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 4, 320)            33341440  
_________________________________________________________________
flatten_3 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1281      
Total params: 33,342,721
Trainable params: 1,281
Non-trainable params: 33,341,440
_________________________________________________________________
None


In [95]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

ValueError: Error when checking input: expected embedding_9_input to have shape (4,) but got array with shape (34549,)

In [90]:
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_train]

In [91]:
# Compute the max lenght of a text
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)
 
N_FEATURES = len(vectorizer.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)
print(X_train_sequences[0])

MAX_SEQ_LENGTH= 34549
[19530 19530 19530 ... 10759  9339 16963]


In [92]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

ValueError: Error when checking input: expected embedding_8_input to have shape (4,) but got array with shape (34549,)

In [65]:
model.fit(padded_docs, labels, epochs=50, verbose=0)
model.evaluate(padded_docs, labels)

3486/3486 [==============================] - 0s 44us/sample - loss: nan - acc: 0.0000e+00


[nan, 0.0]

In [66]:
VALIDATION_SIZE = 500

In [96]:
model.fit(padded_docs[:-VALIDATION_SIZE], labels[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(padded_docs[-VALIDATION_SIZE:], labels[-VALIDATION_SIZE:]))

Train on 2986 samples, validate on 500 samples
Epoch 1/3
2986/2986 [==============================] - 0s 109us/sample - loss: nan - acc: 3.3490e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/3
2986/2986 [==============================] - 0s 62us/sample - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/3
2986/2986 [==============================] - 0s 61us/sample - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00


In [80]:
def encodeY(Y):
    '''create one-hot (dummies) for output, see also https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = tf.keras.utils.to_categorical(encoded_Y)
    return dummy_y

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(df['text'], encodeY(df['topic'].map(int)), test_size = 0.2)

In [85]:
vectorizer = CountVectorizer(min_df=5, max_df=.9)
X_train_onehot = vectorizer.fit_transform(X_train)

In [86]:
numberoflabels = len(set((str(e) for e in y_train.tolist())))
numberoflabels

4

In [87]:
model.fit(X_train_onehot[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=1,
          validation_data=(X_train_onehot[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

ValueError: Error when checking input: expected embedding_7_input to have shape (4,) but got array with shape (19530,)

In [97]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])

NameError: name 'X_test_sequences' is not defined

In [119]:
from gensim.models import word2vec
model = gensim.models.Word2Vec.load('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018')
model.wv.save_word2vec_format('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018.txt', binary=False)

In [128]:
model = KeyedVectors.load('/Users/anne/repos/embedding_models/w2v_adtrouw_1991_2018')

In [116]:
from gensim.models import KeyedVectors

In [140]:
model = KeyedVectors.load('/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15')

In [141]:
model.wv.save_word2vec_format('/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15.txt', binary=False)

In [143]:
embeddings_index = {}
with open('/Users/anne/repos/embedding_models/w2v_model_nr_7_window_10_size_300_negsample_15.txt') as f:
    numberofwordvectors, dimensions = [int(e) for e in next(f).split()]
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
print('Should be {} vectors with {} dimensions'.format(numberofwordvectors, dimensions))

Found 1055100 word vectors.
Should be 1055100 vectors with 300 dimensions


In [144]:
embedding_matrix = np.zeros((N_FEATURES + 1, dimensions))
for i,word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [136]:
embedding_layer = tf.keras.layers.Embedding(N_FEATURES + 1,
                            dimensions,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH,
                            trainable=False)

In [137]:
MAX_SEQ_LENGTH = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGTH=", MAX_SEQ_LENGTH)

MAX_SEQ_LENGTH= 34549


In [138]:
model =  tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=numberoflabels, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 34549, 300)        5859300   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 34545, 64)         96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6909, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 442176)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                28299328  
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 260       
Total params: 34,254,952
Trainable params: 28,395,652
Non-trainable params: 5,859,300
_________________________________

In [139]:
model.fit(X_train_sequences[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train_sequences[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

Train on 2288 samples, validate on 500 samples
Epoch 1/3
2288/2288 [==============================] - 633s 277ms/sample - loss: 1.3651 - acc: 0.3208 - val_loss: 1.2396 - val_acc: 0.3160
Epoch 2/3
1344/2288 [================>.............] - ETA: 3:38 - loss: 1.1894 - acc: 0.3229

KeyboardInterrupt: 

In [113]:
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGTH, value=N_FEATURES)

In [114]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])

698/698 [==============================] - 60s 85ms/sample - loss: 0.6765 - acc: 0.7650
Accuracy: 0.76504296
